# Replication of Community_study.ipynb (Fabrice)

In [ ]:
import sys
sys.path.append('../src/') # adding path to use consensus clustering

In [ ]:
from scipy.io import loadmat
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx

import nilearn
from nilearn.datasets import fetch_atlas_aal
import nilearn.plotting as plotting
import numpy as np

In [ ]:
# load the subject data
G = nx.convert_matrix.from_numpy_matrix(loadmat('../Data/sub1_SC.mat')['sub1_SC'])
# draw the graph
# Important: call this one only once, so that node positions are fixed afterwards 
# (The spring representation does not necessarily reach a global extremum and might lead to visually 
# different graph orientations and confusion conclusions)
pos = nx.spring_layout(G, scale=10) 

In [ ]:
%matplotlib notebook
# compute the best partition
partition = community_louvain.best_partition(G, weight='weight', resolution=0.5)

# color the nodes according to their partition
cmap = cm.get_cmap('hsv', max(partition.values()) + 1)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=40,
                       cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()

### Brain visualization

In [ ]:
aal = fetch_atlas_aal()
# Get coordinates of the regions in AAL atlas and their corresponding labels
coordinates, label_list = plotting.find_parcellation_cut_coords(labels_img=aal['maps'], return_label_names=True) # Note that we compute coordinates for all 116 nodes here, but it doesn't really matter

# Re-order coordinates in sorted order of labels, so that they match with the original atlas' order
coords = coordinates[np.argsort(label_list)]

# We only consider the first 90 regions and ignore cerebellum
limit = 90

def plot_markers_based_on_partition(coords, partition, cmap, output_name='community_example.html'):
    """
    Given markers (defined by their coordinates) as well as a color map function and a partition vector, plot in 
    interactive 3D plot markers in MNI space, overlaid on a glass brain.
    The visualization is saved in the Results subdirectory as an HTML file with the provided name.
    :param coords: 3D coordinates of each marker in MNI space (should be N x 3, where N the number of markers)
    :param partition: Nx1 vector of assignments, denoting for each marker its community
    :param cmap: Colormap function
    :param output_name: Name under which to save visualization. (Default: community_example.html)
    :return: 
    """
    # Plot first 90 region coordinates. Each node is colored according to its community value
    view = plotting.view_markers(coords, cmap(partition)) 
    view.save_as_html('../Results/' + output_name)
    view.open_in_browser()
    return view
    
plot_markers_based_on_partition(coords[:limit], list(partition.values())[:limit], cmap)

## Consensus Clustering

In [ ]:
from consensus_clustering import pairwise_accord_naive, are_equal_up_to_bijection

In [ ]:
def compute_consensus_from_graph(graph, number_partitions, threshold, resolution):
    """
    Applies Louvain algorithm number_partitions times, with provided resolution.
    For each partition, compute the pairwise accordance (ie: if two nodes are grouped in same community or not), and constitute consensus matrix as the sum of these accordance matrices
    The consensus matrix is then normalized by the total number of partitions and thresholded by the provided threshold value.
    
    :param graph: The graph on which to apply the algorithm
    :param number_partitions: Number of different partitions to compute
    :param threshold: Value with which to threshold the consensus matrix, setting all entries below threshold to zero
    :param resolution: Resolution to use in the Louvain algorithm
    
    :return consensus_matrix, the consensus matrix obtained by the procedure
    :return partitions, the number_partitions partitions obtained by the procedure
    """
    n_nodes = len(list(graph.nodes))
    consensus_matrix = np.zeros((n_nodes, n_nodes))
    partitions = np.zeros((number_partitions, n_nodes))
    # First get consensus matrix
    for i in range(0, number_partitions):
        partition = community_louvain.best_partition(graph, weight='weight', resolution=resolution)
        partitions[i, :] =  np.asarray(list(partition.values()))
        consensus_matrix += pairwise_accord_naive(partitions[i, :])
    consensus_matrix /= number_partitions
    # Threshold
    consensus_matrix[consensus_matrix < threshold] = 0.0
    return consensus_matrix, partitions

In [ ]:
def consensus_clustering(graph, number_partitions, threshold, n_steps, resolution):
    """
    Overall consensus clustering algorithm described in https://www.nature.com/articles/srep00336#rightslink
    A first consensus matrix is computed on the original graph.
    Then this consensus matrix is treated as an adjacency matrix itself, on which we apply the consensus procedure until either
    convergence (meaning all partitions are the same) or maximum number of steps are reached.

    :param graph: The original graph on which to perform consensus clustering
    :param number_partitions: Number of partitions at each iteration of the clustering algorithm
    :param threshold: The threshold used in consensus matrix computation
    :param n_steps: The maximum number of steps before termination of the algorithm, should it fail to reach convergence
    :param resolution: The resolution for Louvain's algorithm

    :return partitions: The last number_partitions derived by the algorithm. If algorithm is converged, they are all equal up to bijection.
    :return i: The iteration at which the algorithm finished. Useful to assess how quickly it converged or if it even converged at all.
    """
    consensus_matrix, partitions = compute_consensus_from_graph(graph, number_partitions, threshold, resolution)

    # Until convergence or number of steps exceeded
    for i in range(0, n_steps):
        # Convert consensus matrix to graph
        G = nx.convert_matrix.from_numpy_matrix(consensus_matrix)
        # Get new consensus matrix
        consensus_matrix, partitions = compute_consensus_from_graph(G, number_partitions, threshold, resolution)

        # Now we must ask if all category vectors are the same or not
        should_stop = True
        for vi in range(0, number_partitions - 1):
            should_stop = are_equal_up_to_bijection(partitions[vi], partitions[vi + 1])
            if not should_stop:
                break
        if should_stop:
            break
    return partitions, i

In [ ]:
partitions, i = consensus_clustering(G, 100, 0.1, 500, 0.4)

In [ ]:
partitions

In [ ]:
i

In [ ]:
p = partitions[0].astype(int)
cmap = cm.get_cmap('hsv', max(p) + 1)

In [ ]:
plot_markers_based_on_partition(coords[:limit], p, cmap)

## Functional graph definition

In [ ]:
FC = loadmat('../Data/sub1_Motor.mat')['sub1_Motor']

In [ ]:
def pearsonr(x):
    """
    Simply computes correlation between all pairs of lines in X.
    It is assumed that X is in the form variables x observations.
    If X is in observations x variables it should be transposed before being passed to this function!
    """
    return np.corrcoef(x, rowvar=True)

def sliding_window_corr(x, window_length, window_stride, window_function):
    """
    
    """
    half_window=window_length//2
    window_f = window_function(window_length) # This serves to taper the samples
    start = half_window
    end = x.shape[0] - half_window
    
    n_windows = (end-start)//window_stride + 1 # Check that this is correct
    
    print('There should be {} windows'.format(n_windows))
    print('Start: {} - End: {}'.format(start, end))
    
    correlations = np.zeros((n_windows,x.shape[1], x.shape[1]))
    for i in range(0, n_windows):
        # The window is centered at i*window_stride+start
        center = i*window_stride + start
        # Left side is center - half_window
        # Right side is center + half_window
        # We compute correlation of current sample tampered with window function
        correlations[i]=pearsonr(x[center-half_window:center+half_window].T*window_f)
    return correlations

In [ ]:
corrs = sliding_window_corr(FC, 60, 1, lambda x: np.hamming(x))

In [ ]:
corrs.shape

In [ ]:
figure=plt.figure(dpi=100)

plt.imshow(corrs[10,:,:], cmap="jet")
plt.show()
plt.colorbar()


In [ ]:
def create_communities_from_partition(G, partition):
    tmp_G = G.copy()
    unique= np.unique(partition)
    for i in range(0, p.size):
        tmp_G.add_nodes_from([i], partition=p[i])
    subgraphs = [tmp_G.subgraph((node for node, data in tmp_G.nodes(data=True) if data.get("partition") == e)) for i, e in enumerate(unique)]
    if not nx.community.is_partition(tmp_G, subgraphs):
        raise AssertionError('The provided partition is not a proper partition for this graph')    
    return subgraphs

In [ ]:
# Assign to every node in the graph its community
subgraphs = create_communities_from_partition(G, p)

# The graph G should not have a partition field, because the method performs its operations on a copy of G
G.nodes[0]

# Now, we are ready to compute some informations on each community. In particular, we can compute for each node its clustering coefficient within its community

### Segregation & Integration

In [ ]:
from functools import reduce


def compute_degree(G, standardize=True, weighted=True):
    """
    Function to compute degree of all nodes within the graph of interest.
    If weighted degree is requested, the degree will then be defined as the sum of weighted edges, for each node.
    If z-scoring is requested, the function will standardize the returned degrees.
    
    :param G: input graph. Degree will be computed for each node within this graph
    :param standardize: Whether degrees should be standardized before being returned. (Default = True)
    :param weighted: Computes weighted degree instead of regular degree. If the graph is unweighted, this parameter is ignored. (Default = True)
    
    :return nodes: Nodes of the graph (for debug purposes)
    :return degrees: Degrees computed according to the chosen method.
    """
    if weighted and nx.is_weighted(G):
        nodes = list(G.nodes)
        degrees = [0]*len(nodes)
        i = 0
        for n in nodes:
            for neighbor in G[n]:
                degrees[i] += G[n][neighbor]['weight']
            i+=1
    else:
        nodes,degrees=zip(*list(G.degree))
    
    degrees = np.asarray(degrees)
    if standardize:
        degrees = (degrees-degrees.mean())/degrees.std()
    return nodes, degrees

def compute_participation_coefficient(G, weighted, partition_values):
    """
    Computes participation coefficient as 1 - sum((k_is / k_i)**2), where k_is is the degree of node i in community s
    and k_i is the degree of node i in the graph (thus, sum of k_is).
    
    :param G: graph on which to compute participation coefficient node-wise.
    :param weighted: Whether to compute weighted degree or not.
    :param partition_values: Vector indicating which community which node belongs to.
    
    :return participation coefficient vector of shape n_nodes x 1
    """
    nodes = list(G.nodes)
    n_nodes = len(nodes)
    partition_categories = np.unique(partition_values)
    degrees = np.zeros((n_nodes, partition_categories.size))
    for n_i in range(0, n_nodes):
        for c_i, u in enumerate(unique_values):
            neighbours = G[n_i]
            for neighbour in neighbours:
                if p[neighbour] == u:
                    if weighted:
                        degrees[n_i, c_i] += neighbours[neighbour]['weight']
                    else:
                        degrees[n_i, c_i] += 1
    norm_factor = 1./ degrees.sum(axis=1)
    s = (norm_factor.reshape((-1,1)) * degrees)**2
    return 1-s.sum(axis=1)
    #return degrees
    

def compute_system_segregation(G, partition_values):
    """
    System segregation is defined as: 
    (mean(connections within same community) - mean(connections not part of same community))/mean(connection within same community)
    """
    # First, compute all communities
    subgraphs = create_communities_from_partition(G, partition_values)
    
    # Next for all communities, compute the within weighted degree values
    degree_values = [compute_degree(subgraph,standardize=False, weighted=True)[1] for subgraph in subgraphs]
    
    # Now we must fuse all these together
    degree_values = reduce(lambda x,y: np.hstack((x, y)), degree_values)

    mean_same_community_weight = degree_values.mean()
    
    # Now, we can worry about node pairs, ie: edges!
    edges = list(G.edges)
    count=0
    mean_diff_community_weight = 0;
    for e in edges:
        n1, n2 = e[0], e[1]
        if partition_values[n1] != partition_values[n2]:
            count += 1
            mean_diff_community_weight += G[n1][n2]['weight']
    mean_diff_community_weight /= count
    #return mean_same_community_weight
    return (mean_same_community_weight - mean_diff_community_weight)/mean_same_community_weight

In [ ]:
compute_system_segregation(G, p).mean()